# About - we will excute the model properly without deployment inside code website is in different file

In [1]:
!ls ../input/training-t5-model-yet-again/outputs 

simplet5-epoch-0-train-loss-1.9906-val-loss-1.8737
simplet5-epoch-1-train-loss-1.7323-val-loss-1.8841
simplet5-epoch-2-train-loss-1.5606-val-loss-1.9126
simplet5-epoch-3-train-loss-1.4229-val-loss-1.9572
simplet5-epoch-4-train-loss-1.3095-val-loss-2.0107
simplet5-epoch-5-train-loss-1.2103-val-loss-2.0811


In [2]:
!pip install simplet5

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 9.5 MB/s eta 0:00:00:00:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.7/527.7 KB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 952.4/952.4 KB 50.6 MB/s eta 0:00:00
  Created wheel for simplet5: filename=simplet5-0.1.4-py3-none-any.whl size=6873 sha256=20e5853b6888dfb3a1999ba5dffcd916018d3d4efabf4085643362cca4f46126
  Stored in directory: /root/.cache/pip/wheels/30/1b/96/47ad5fda68def6635961fd0ca312fc0ce3670936ae09d97c43
Successfully built simplet5
  Attempting uninstall: setuptools
    Found existing installation: setuptools 59.8.0
    Uninstalling setuptools-59.8.0:
      Successfully uninstalled setuptools-59.8.0
  Attempting uninstall: pyDeprecate
    Found existing installation: pyDeprecate 0.3.2
    Uninstalling pyDeprecate-0.3.2:
      Successfully uninstalled pyDeprecate-0.3.2
  Attempting uninstall: transformers
    Found existing installation: transfor

# Load model

In [3]:
model2="../input/training-t5-model-yet-again/outputs/simplet5-epoch-1-train-loss-1.7323-val-loss-1.8841" 
# import
from simplet5 import SimpleT5

# instantiate
model = SimpleT5()

# load (supports t5, mt5, byT5 models)

model.load_model("t5",model2,use_gpu=True)

# Your paragraph

In [4]:
para="""
Oxygen is the chemical element with the symbol O and atomic number 8. It is a member of the chalcogen group in the periodic table, a highly reactive nonmetal, and an oxidizing agent that readily forms oxides with most elements as well as with other compounds. Oxygen is Earth's most abundant element, and after hydrogen and helium, it is the third-most abundant element in the universe. At standard temperature and pressure, two atoms of the element bind to form dioxygen, a colorless and odorless diatomic gas with the formula O
2. Diatomic oxygen gas currently constitutes 20.95% of the Earth's atmosphere, though this has changed considerably over long periods of time. Oxygen makes up almost half of the Earth's crust in the form of oxides.[3]
"""

# Here comes our sliding window approach

In [5]:
class SlidingWindow():
    def __init__(self,words,stride,window):
        self.idx1=0 
        self.idx2=window 
        self.stride=stride
        self.words=words
    def get_item(self):
        if self.idx2-len(self.words)>=self.stride:
            return "END"
        output=self.words[self.idx1:min([len(self.words),self.idx2])] 
        self.idx1+=self.stride 
        self.idx2+=self.stride  
        return output 
slided=SlidingWindow([1,2,3,4,5,6,7,8,9],3,4) 
        
        

In [6]:
slided.get_item()

[1, 2, 3, 4]

In [7]:
slided.get_item()

[4, 5, 6, 7]

In [8]:
slided.get_item()

[7, 8, 9]

In [9]:
slided.get_item()

'END'

# Generate Questions 

## Loading pretrained model for answers

In [10]:
from transformers import pipeline 
nlp = pipeline("question-answering") 


No model was supplied, defaulted to distilbert-base-cased-distilled-squad (https://huggingface.co/distilbert-base-cased-distilled-squad)


Downloading:   0%|          | 0.00/473 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/249M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

In [13]:
def create_questions(text,number,stride_percent=70):
    words=text.split()  
    window_size=int(len(words)/number)
    stride=max([2,int(window_size*stride_percent/100)])
    slided=SlidingWindow(words,stride,window_size) 
    questions=[]
    answers=[]
    while True: 
        item=slided.get_item()
        if item=="END":
            break 
        questions+=model.predict(" ".join(item))
        answers.append(nlp(question=questions[-1],context=text)['answer'])
    return questions,answers
create_questions(para,5,80)

(['What group does Oxygen belong to?',
  'What is a highly reactive nonmetal?',
  'What is the third most abundant element in the world?',
  'At standard temperature and pressure, how many atoms of the element bind to form dioxygen?',
  'What formula does diatomic oxygen gas have?',
  "How much of the Earth's crust is made up of oxides?"],
 ['chalcogen', 'Oxygen', 'Oxygen', 'two', 'O\n2', 'almost half'])